In [1]:
import os
import cv2
import random
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Reshape

In [ ]:
def process_dataset(images_dir, annotations_dir):
    images = []
    bounding_boxes = []

    for filename in os.listdir(images_dir):
        if filename.endswith('.png'):
            # Baca gambar
            img_path = os.path.join(images_dir, filename)
            image = cv2.imread(img_path)

            # Ubah ukuran gambar
            resized_image = cv2.resize(image, (1280, 720))

            # Normalisasi gambar
            normalized_image = resized_image / 255.0

            # Tambahkan gambar ke daftar
            images.append(normalized_image)

            # Proses file XML
            xml_filename = filename.replace('.png', '.xml')
            xml_path = os.path.join(annotations_dir, xml_filename)

            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Daftar untuk menyimpan bounding box untuk gambar
            boxes = []

            for obj in root.findall('object'):
                bbox = obj.find('bndbox')
                xmin = int(bbox.find('xmin').text)
                ymin = int(bbox.find('ymin').text)
                xmax = int(bbox.find('xmax').text)
                ymax = int(bbox.find('ymax').text)

                # Simpan bounding box ke daftar dengan confidence score 1
                boxes.append([xmin, ymin, xmax, ymax, 1])  # Tambahkan confidence score

            bounding_boxes.append(boxes)

    # Konversi list ke numpy array untuk digunakan sebagai input model
    images = np.array(images)
    bounding_boxes = np.array(bounding_boxes, dtype=object)  # Gunakan dtype=object untuk array dengan ukuran berbeda

    return images, bounding_boxes


In [ ]:
# Path ke dataset
train_images_dir = './tes/20m/train'
train_annotations_dir = './tes/20m/train/annotation'

validation_images_dir = './tes/20m/val'
validation_annotations_dir = './tes/20m/val/annotation'

test_images_dir = './tes/20m/test'
test_annotations_dir = './tes/20m/test/annotation'

In [ ]:
# Proses dataset latih
X_train, Y_train = process_dataset(train_images_dir, train_annotations_dir)

# Proses dataset validasi
X_val, Y_val = process_dataset(validation_images_dir, validation_annotations_dir)

# Proses dataset uji
X_test, Y_test = process_dataset(test_images_dir, test_annotations_dir)

In [ ]:
input_shape = (720, 1280, 3)

inputs = Input(shape=input_shape)

# Convolutional Layers
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)

# Dense Layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# Output Layer untuk prediksi bounding boxes
# Gantilah Dense layer ini dengan layer yang sesuai untuk jumlah bounding boxes yang variatif
output_layer = Dense(5 * 50, activation='linear')(x)  # Misalkan 50 adalah jumlah bounding boxes maksimum yang diharapkan

# Reshape output ke (jumlah_bounding_boxes, 5)
output_layer = Reshape((50, 5))(output_layer)

model = Model(inputs=inputs, outputs=output_layer)

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


In [ ]:
# Tes
idx = random.randint(0, len(X_train) - 1)
image_to_show = X_train[idx]
plt.imshow(image_to_show)
plt.show()

print("X_train : ", len(X_train))
print("Y_train : ", len(Y_train))

print("X_val : ", len(X_val))
print("Y_val : ", len(Y_val))

print("X_test : ", len(X_test))
print("Y_test : ", len(Y_test))
